<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/BlockChain/python_blockchain_training_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!mkdir utility 

mkdir: cannot create directory ‘utility’: File exists


In [29]:
pip install  pycrypto

     |████████████████████████████████| 450kB 4.8MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp36-cp36m-linux_x86_64.whl size=499958 sha256=627ff363eab9ed6d0ddb5ac0d0c40003acd6ddce39d99a7aeb85f40abc0df2de
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


In [33]:
%%writefile utility/__init__.py
'''package'''
from utility.hash_util import hash_string_256

__all__ = ['hash_string_256']

Overwriting utility/__init__.py


In [34]:
%%writefile utility/hash_util.py
import hashlib as hl
import json

#__all__ = ['hash_string_256','hash_block']

def hash_string_256(string):
  return hl.sha256(string).hexdigest()

def hash_block(block):
  #print(block)
  hashable_block = block.__dict__.copy()
  hashable_block['transactions'] = [tx.to_ordered_dict() for tx in hashable_block['transactions']]
  return hash_string_256(json.dumps(hashable_block, sort_keys=True).encode())


Overwriting utility/hash_util.py


In [35]:
%%writefile utility/verification.py
""" Provides verification helper methods."""

from utility.hash_util import hash_string_256, hash_block

class Verification:
  """A helper class which offer various static and class-based veriffication"""
  @staticmethod
  def valid_proof(transactions, last_hash, proof):
    guess = (str([tx.to_ordered_dict() for tx in transactions]) + str(last_hash) + str(proof)).encode()
    guess_hash = hash_string_256(guess)
    #print(guess_hash)
    return guess_hash[0:2] == '00'

  @classmethod
  def verify_chain(cls, blockchain):
    """verify current blockchain"""
    for (index, block) in enumerate(blockchain):
      if index == 0:
        continue
      if block.previous_hash != hash_block(blockchain[index -1]):
        return False
      if not cls.valid_proof(block.transactions[:-1], block.previous_hash, block.proof):
        print('Prood of work invalid')
        return False
    return True

  @staticmethod
  def verify_transaction(transaction, get_balance):
    sender_balance = get_balance()
    return sender_balance >= transaction.amount

  @classmethod
  def verify_transactions(cls, open_transactions, get_balance):
    return all([cls.verify_transaction(tx, get_balance) for tx in open_transactions])


Overwriting utility/verification.py


In [36]:
%%writefile utility/printable.py
class Printable:
  def __repr__(self):
    return str(self.__dict__)


Overwriting utility/printable.py


In [48]:
%%writefile wallet.py
from Crypto.PublicKey import RSA
import Crypto.Random
import binascii 

class Wallet:
  def __init__(self):
    self.private_key = None
    self.public_key = None

  def create_keys(self):
    private_key, public_key = self.generate_keys()
    self.private_key = private_key
    self.public_key = public_key 

  def load_keys(self):
      pass

  def generate_keys(self):
    private_key = RSA.generate(1024,Crypto.Random.new().read)
    public_key = private_key.publickey()
    return (binascii.hexlify(private_key.exportKey(format='DER')).decode('ascii'),
            binascii.hexlify(public_key.exportKey(format='DER')).decode('ascii'))

Overwriting wallet.py


In [49]:
%%writefile node.py
from uuid import uuid4
from blockchain import Blockchain
from utility.verification import Verification
from wallet import Wallet


class Node:
  def __init__(self):
    self.wallet = Wallet()
    #self.id = str(uuid4())
    self.blockchain = Blockchain(self.wallet.public_key)
    

  #using function for reusable code.
  def get_transaction_value(self):
    """ Retruns the input of the user(a new transaction amount) as a float
    """
    tx_recipient = input('Enter the recipent of the transaction: ') 
    tx_amount = float(input('Your trasaction amount please: ')) 
    return (tx_recipient, tx_amount)
  
  def get_user_choice(self):
    user_input = input('Your Choice : ')
    return user_input

  def print_blockchain_elements(self):
    #use for loop to output the blockchain list console
    for block in self.blockchain.chain:
      print('outputting block : ', block)
    else:
      print('-' * 20)

  def listen_for_input(self):
    waiting_for_input = True
    while waiting_for_input:
      print('Please choose')
      print('1: Add a new transaction value')
      print('2: Mine a new block')
      print('3: Output the blockchain blocks')
      print('4: check transaction validity')
      print('5: Create wallet')
      print('6: load wallet')
      print('q : Quit ')
      user_choice = self.get_user_choice()
      if user_choice == '1':
        tx_data = self.get_transaction_value()
        recipient, amount = tx_data
        if self.blockchain.add_transaction(recipient, self.wallet.public_key, amount=amount):
          print('Added transaction')
        else:
          print('Tranaction failed')
        print(self.blockchain.get_open_transactions())
      elif user_choice == '2':
        self.blockchain.mine_block()
      elif user_choice == '3':
          self.print_blockchain_elements()
      elif user_choice == '4':
        if Verification.verify_transactions(self.blockchain.get_open_transactions(), self.blockchain.get_balance):
          print("All Transactions are valid")
        else:
          print('There are invalid transaction')
      elif user_choice == '5':
        self.wallet.create_keys()
      elif user_choice == '6':
        pass
      elif user_choice == 'q':
        waiting_for_input = False
      else:
        print('Input was invalid, please pick a value from the list!')    
      if not Verification.verify_chain(self.blockchain.chain):
        self.print_blockchain_elements()
        print('Invalid block chain!')
        break
      print('Balance of {} : {:6.2f}'.format(self.wallet.public_key, self.blockchain.get_balance()))  
      print('Choice registered !')
    else:
      print('User left')
    print('Done!') 

if __name__ == '__main__':
  node = Node()
  node.listen_for_input() 

#print(__name__)


Overwriting node.py


In [50]:
%%writefile block.py
from time import time
from utility.printable import Printable

class Block(Printable):
  def __init__(self, index, previous_hash, transactions, proof, time=time()):
    self.index = index
    self.previous_hash = previous_hash
    self.transactions = transactions
    self.proof = proof
    self.time = time


Overwriting block.py


In [51]:
%%writefile transaction.py
from collections import OrderedDict
from utility.printable import Printable

class Transaction(Printable):
  def __init__(self, sender, recipient, amount):
    self.sender = sender
    self.recipient = recipient
    self.amount = amount



  def to_ordered_dict(self):
      return OrderedDict([('sender', self.sender),
                          ('recipient', self.recipient),
                          ('amount', self.amount)])

Overwriting transaction.py


In [52]:
%%writefile blockchain.py
# try except block : 
#####################################################
import functools
import hashlib as hl
import json
from collections import OrderedDict
import sys

from utility.hash_util import  hash_block
from block import Block
from transaction import Transaction
from time import time
from utility.verification import Verification

#global variable
MINING_REWARD = 10

#print(__name__)

class Blockchain:
  def __init__(self, hosting_node_id):
    genesis_block = Block(index = 0,
                        previous_hash= '',
                        transactions = [],
                        proof = 100,
                        time = 0)

    self.chain = [genesis_block]
    self._open_transactions = []
    self.hosting_node = hosting_node_id
    owner = 'Sumith'
    participants = {owner}
    self.load_data()

  @property
  def chain(self):
      return self._chain[:]
  
  @chain.setter
  def chain(self, val):
    self._chain = val

  def get_open_transactions(self):
    return self._open_transactions[:]

  def load_data(self):
    try:
      with open('blockchain.txt', mode='r') as f:
        file_content = f.readlines()

        blockchain = json.loads(file_content[0][:-1])
        updated_blockchain=[]
        for block in blockchain:
          converted_tx = [Transaction(tx['sender'],
                                      tx['recipient'],
                                      tx['amount']) for tx in block['transactions']]
          updated_block = Block(block['index'],
                                block['previous_hash'],
                                converted_tx,
                                block['proof'],
                                block['time'])
          updated_blockchain.append(updated_block)
        self.chain = updated_blockchain
      
        open_transactions = json.loads(file_content[1])
        updated_transactions = []
        for tx in open_transactions:
          updated_transaction = Transaction(tx['sender'],
                                             tx['recipient'],
                                             tx['amount'])
          updated_transactions.append(updated_transaction) 
        self._open_transactions = updated_transactions
    except (IOError, IndexError):                    
      pass
    except ValueError:
      print('Value error!')
    except Exception as exc:
      print("Oops!",exc)
      print("Oops!",sys.exc_info()[0],"occured.")
      sys.exit(1)
    finally:
      print('Cleanup!')

  def save_data(self):
    try :
      with open('blockchain.txt', mode='w') as f:
        saveable_chain = [block.__dict__ for block in [Block(
            block_el.index,
            block_el.previous_hash,
            [tx.__dict__ for tx in block_el.transactions],
            block_el.proof,
            block_el.time
            ) for block_el in self._chain]]

        f.write(json.dumps(saveable_chain))
        f.write('\n')
        saveable_tx = [tx.__dict__ for tx in self._open_transactions]
        f.write(json.dumps(saveable_tx))
    except IOError:
      print('Error saving file!')

  def proof_of_work(self):
    last_block = self._chain[-1]
    last_hash = hash_block(last_block)
    proof = 0
    while not Verification.valid_proof(self._open_transactions, last_hash, proof):
      proof += 1
    return proof

  def get_balance(self):
    participant = self.hosting_node
    tx_sender = [[tx.amount for tx in block.transactions if tx.sender == participant] for block in self._chain]
    open_tx_sender = [tx.amount for tx in self._open_transactions if tx.sender == participant]
    tx_sender.append(open_tx_sender)
    amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
    tx_recipient = [[tx.amount for tx in block.transactions if tx.recipient == participant] for block in self._chain]
    amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
    return  amount_received - amount_sent


  def get_last_blockchain_value(self):
    """ returns the last value of the current bloackchain """
    #function to return the value
    if len(self._chain) < 1 :
      return None
    return self._chain[-1]

  def add_transaction(self, recipient,sender, amount=1.0):
    transaction = Transaction(sender, recipient, amount)
    if Verification.verify_transaction(transaction, self.get_balance):
      self._open_transactions.append(transaction)
      self.save_data()
      return True
    return False

  def mine_block(self):
    last_block = self._chain[-1]
    hashed_block = hash_block(last_block)
    proof = self.proof_of_work() 
    reward_transaction = Transaction(sender = 'MINING', recipient = self.hosting_node, amount = MINING_REWARD)
    copied_transactions = self._open_transactions[:]
    copied_transactions.append(reward_transaction)
    block = Block(index = len(self._chain),
                        previous_hash = hashed_block,
                        transactions = copied_transactions,
                        proof = proof,
                        time = 0)
    self._chain.append(block)
    self._open_transactions = []
    self.save_data()
    return True




Overwriting blockchain.py


In [53]:
!python node.py

Cleanup!
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
5: Create wallet
6: load wallet
q : Quit 
Your Choice : 2
Balance of None :  10.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
5: Create wallet
6: load wallet
q : Quit 
Your Choice : 3
outputting block :  {'index': 0, 'previous_hash': '', 'transactions': [], 'proof': 100, 'time': 0}
outputting block :  {'index': 1, 'previous_hash': 'f43bf96b7d7071552937a9fa0a82076d92554d2d593eaf76d8e77f19bd7cf5b8', 'transactions': [{'sender': 'MINING', 'recipient': None, 'amount': 10}], 'proof': 195, 'time': 0}
--------------------
Balance of None :  10.00
Choice registered !
Please choose
1: Add a new transaction value
2: Mine a new block
3: Output the blockchain blocks
4: check transaction validity
5: Create wallet
6: load wallet
q : Quit 
Your Choice : q
Balance of

In [0]:
ls -rlt utility/

total 20
-rw-r--r-- 1 root root   13 Mar 12 17:46 __init__.py
-rw-r--r-- 1 root root  359 Mar 12 17:46 hash_util.py
-rw-r--r-- 1 root root 1096 Mar 12 17:46 verification.py
-rw-r--r-- 1 root root   68 Mar 12 17:46 printable.py
drwxr-xr-x 2 root root 4096 Mar 12 17:46 __pycache__/


In [0]:
ls -rlt

total 36
drwxr-xr-x 1 root root 4096 Mar  3 18:11 sample_data/
drwxr-xr-x 3 root root 4096 Mar 12 17:43 utility/
-rw-r--r-- 1 root root 2484 Mar 12 17:46 node.py
-rw-r--r-- 1 root root  307 Mar 12 17:46 block.py
-rw-r--r-- 1 root root  428 Mar 12 17:46 transaction.py
-rw-r--r-- 1 root root 5225 Mar 12 17:46 blockchain.py
drwxr-xr-x 2 root root 4096 Mar 12 17:46 __pycache__/
-rw-r--r-- 1 root root  541 Mar 12 17:47 blockchain.txt


In [0]:
!rm blockchain.txt

In [0]:
!cat blockchain.txt

[{"index": 0, "previous_hash": "", "transactions": [], "proof": 100, "time": 0}, {"index": 1, "previous_hash": "f43bf96b7d7071552937a9fa0a82076d92554d2d593eaf76d8e77f19bd7cf5b8", "transactions": [{"sender": "MINING", "recipient": "Sumith", "amount": 10}], "proof": 195, "time": 0}, {"index": 2, "previous_hash": "a2e61e99c6c2d59fd55a172f4e8bda4eb57ed11eb95b15acca0aa2409b78a953", "transactions": [{"sender": "Sumith", "recipient": "we", "amount": 1.0}, {"sender": "MINING", "recipient": "Sumith", "amount": 10}], "proof": 638, "time": 0}]
[]

In [0]:
2